In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [2]:
#create single image .pt file
import torch
from PIL import Image
import torchvision.transforms as transforms

# Define the path to your single image
img_path = "/Users/annastuckert/Documents/GitHub/DeepEnsampleGUI/napari-threshold/src/napari_threshold/frame_0002.jpg"  # Replace with your image path

# Define any transformations (optional, for example, converting to tensor)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
])

# Load the single image
img = Image.open(img_path).convert('L')  # Convert to grayscale ('L')

# Resize the image
resize_factor = 0.2
min_size = 32
new_width = max(int(img.width * resize_factor), min_size)
new_height = max(int(img.height * resize_factor), min_size)

# Ensure the resized dimensions are even
new_width += new_width % 2  # Make width even
new_height += new_height % 2  # Make height even

img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

# Apply the transformation (e.g., converting to tensor)
img_tensor = transform(img)

# Define the save path for the .pt file
save_path = '/Users/annastuckert/Documents/GitHub/DeepEnsampleGUI/napari-threshold/src/napari_threshold/single_image.pt'  # Change to the path where you want to save the file

# Save the image tensor to the .pt file
torch.save(img_tensor, save_path)
print(f"Image saved to {save_path}")

Image saved to /Users/annastuckert/Documents/GitHub/DeepEnsampleGUI/napari-threshold/src/napari_threshold/single_image.pt


In [3]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF
import random

class facemapdataset(Dataset):
    def __init__(self, data_file='/Users/annastuckert/Documents/GitHub/DeepEnsampleGUI/napari-threshold/src/napari_threshold/single_image.pt', transform=None):
        super().__init__()
        self.transform = transform
        # Load the data (single image tensor)
        self.data= torch.load(data_file)

    def __len__(self):
        return len(self.data)  # We are using a single image dataset, no augmentation.

    def __getitem__(self, index):
        # Load the image and label
        image = self.data[index].clone()

        # No augmentation or transformation is applied to the image or label
        return image


# Initialize the dataset
dataset = facemapdataset()

# Example of retrieving a sample from the dataset
image = dataset[0]
print(f"Image shape: {image.shape}")

Image shape: torch.Size([244, 388])


In [4]:
import torch
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from models import Unet


# Initialize the Unet model
model = Unet()

# Load the model weights
model.load_state_dict(torch.load('/Users/annastuckert/Documents/GitHub/DeepEnsampleGUI/napari-threshold/src/napari_threshold/model_weights.pth', map_location=device))
model.eval()  # Set the model to evaluation mode




Unet(
  (uEnc11): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (uEnc12): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (uEnc2): convBlock(
    (enc1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (enc2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (scale): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (act): ReLU()
  )
  (uEnc3): convBlock(
    (enc1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (enc2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (scale): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (act): ReLU()
  )
  (dec3): convBlock(
    (enc1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (enc2): Conv2d(16, 16, kernel_size=(3, 3), 

In [5]:

# Assuming image is already loaded
image = dataset[0]  # Shape: [244, 388]
image = image.unsqueeze(0).unsqueeze(0)  # Now the shape is [1, 1, 244, 388]

# Move the image to the device (GPU or CPU)
inputs = image.to(device)

# Perform inference with the model
with torch.no_grad():
    scores, fmap = model(inputs)

    # Optionally, process the results (e.g., loss, visualization)
    img = inputs.squeeze().detach().cpu().numpy()
    pred = scores.squeeze().detach().cpu().numpy()
    fmap = fmap.mean(1).squeeze().cpu().numpy()

    # Plotting the results
    plt.clf()
    plt.figure(figsize=(12, 4))
    plt.subplot(141)
    plt.imshow(img, cmap="gray")
    plt.subplot(143)
    plt.imshow(pred)
    plt.subplot(144)
    plt.imshow(fmap)

    plt.tight_layout()
    plt.savefig("test_%03d.jpg")
    plt.close()

<Figure size 640x480 with 0 Axes>